<a target="_blank" href="https://colab.research.google.com/github/gox6/colab-demos/blob/main/wip/polars-a-modern-dataframe-library.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

<html>
<body>
    <div style="display: flex;">
        <a href="https://towardsdatascience.com/evaluate-rags-rigorously-or-perish-54f790557357" style="width: 45em; height: 15em; background-color: white; padding-left: 1em; display: flex; flex-direction: column; justify-content: center; color: inherit; text-decoration: none;">
            <p style="margin: 0; font-size: 1.5em; color: darkblue; font-weight: bold;">The notebook contains the code for Medium article</p>
            <p style="margin: 0; font-weight: bold; font-size: 1.5em; color: #3D4849;">Invent the title</p>
        </a>
        <a href="https://towardsdatascience.com/evaluate-rags-rigorously-or-perish-54f790557357" style="width: 30em; height: 15em;">
            <img src="https://images.unsplash.com/photo-1522991150037-09a660b94ba4?q=80&w=1470&auto=format&fit=crop&ixlib=rb-4.0.3&ixid=M3wxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8fA%3D%3D" style="width: 100%; height: 100%; object-fit: cover;">
        </a>
    </div>
</body>
</html>

# Project Setup

In [2]:
# Installing Python packages quietly

!pip install --quiet \
  polars \
  1> /dev/null

In [3]:
# Importing the packages
from getpass import getpass
import os

import polars as pl

In [4]:
# Managing secrets
# - If using Colab please use Colab Secrets
# - If running outside Colab please provide secrets as environmental variables
COLAB = os.getenv("COLAB_RELEASE_TAG") is not None

if COLAB:
    from google.colab import userdata, data_table

    # Secrets
    OPENAI_API_KEY = userdata.get("OPENAI_API_KEY")
    os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
    runtime_info = "Colab runtime"

    # Enabling Colab's data formatter for pandas
    data_table.enable_dataframe_formatter()
elif OPENAI_API_KEY := os.environ.get("OPENAI_API_KEY"):
    # Secrets
    runtime_info = "Non Colab runtime"
else:
    OPENAI_API_KEY = getpass("OPENAI_API_KEY")
    os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
    runtime_info = "Non Colab runtime"

print(runtime_info)

Colab runtime


In [5]:
# You need free tier GCP account
from google.colab import auth
auth.authenticate_user()

In [6]:
# Perform a query.
%%time
from google.cloud import bigquery

client = bigquery.Client(project='handy-buttress-422914-t8')
QUERY = (
    """SELECT * FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
    LIMIT 10000000"""
    )
query_job = client.query(QUERY)  # API request
rows = query_job.result()  # Waits for query to finish

df_pl = pl.from_arrow(rows.to_arrow())

CPU times: user 25 s, sys: 12.4 s, total: 37.4 s
Wall time: 46.9 s


In [7]:
df_pl.describe()

describe,unique_key,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_census_tract,dropoff_census_tract,pickup_community_area,dropoff_community_area,fare,tips,tolls,extras,trip_total,payment_type,company,pickup_latitude,pickup_longitude,pickup_location,dropoff_latitude,dropoff_longitude,dropoff_location
str,str,str,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,str,f64,f64,str,f64,f64,str
"""count""","""10000000""","""10000000""","""10000000""","""9999570""",9.995447e6,9.999831e6,6.332347e6,6.334679e6,8.399841e6,8.31957e6,9.999701e6,9.999701e6,7.826915e6,9.999701e6,9.999701e6,"""10000000""","""7038257""",8.400944e6,8.400944e6,"""8400944""",8.343984e6,8.343984e6,"""8343984"""
"""null_count""","""0""","""0""","""0""","""430""",4553.0,169.0,3.667653e6,3.665321e6,1.600159e6,1.68043e6,299.0,299.0,2.173085e6,299.0,299.0,"""0""","""2961743""",1.599056e6,1.599056e6,"""1599056""",1.656016e6,1.656016e6,"""1656016"""
"""mean""",null,null,null,null,830.940642,3.951144,1.7031e10,1.7031e10,23.85362,22.08319,13.29451,1.409008,0.007202,0.990582,15.761887,null,null,41.901105,-87.659719,null,41.901224,-87.655395,null
"""std""",null,null,null,null,1463.502409,17.236593,344129.45913,335766.338394,19.953093,18.141551,47.74395,2.721428,0.877186,11.775439,50.206026,null,null,0.038236,0.073829,null,0.038818,0.061827,null
"""min""","""00000083123c59…","""0005e9157e3cfe…","""2013-01-01 00:…","""2013-01-01 00:…",0.0,0.0,1.7031e10,1.7031e10,1.0,1.0,0.0,0.0,0.0,0.0,0.0,"""Cash""","""0118 - 42111 G…",41.660136,-87.913625,"""POINT (-87.534…",41.660136,-87.913625,"""POINT (-87.531…"
"""25%""",null,null,null,null,360.0,0.7,1.7031e10,1.7031e10,8.0,8.0,6.0,0.0,0.0,0.0,7.0,null,null,41.880994,-87.655879,null,41.880994,-87.655998,null
"""50%""",null,null,null,null,556.0,1.5,1.7031e10,1.7031e10,24.0,22.0,8.05,0.0,0.0,0.0,9.6,null,null,41.892073,-87.632746,null,41.892508,-87.632746,null
"""75%""",null,null,null,null,960.0,3.6,1.7032e10,1.7032e10,32.0,32.0,13.45,2.0,0.0,1.0,15.3,null,null,41.907492,-87.626211,null,41.914747,-87.626211,null
"""max""","""ffffff7320d385…","""fff84aa08ac788…","""2023-12-30 23:…","""2023-12-30 23:…",86398.0,3353.1,1.7032e10,1.7032e10,77.0,77.0,9715.7,881.0,999.99,7789.97,9716.7,"""Way2ride""","""Yellow Cab""",42.021224,-87.534903,"""POINT (-87.913…",42.021224,-87.531386,"""POINT (-87.913…"


In [8]:
%%time
results_pl = (
    df_pl
    .filter(pl.col("trip_miles") <= pl.lit(5))
    .group_by("payment_type")
    .agg(pl.col("fare").mean())
)

CPU times: user 4 s, sys: 5.95 s, total: 9.95 s
Wall time: 5.38 s


In [9]:
df_pd = df_pl.to_pandas()


In [10]:
%%time
results_pd = df_pd[df_pd["trip_miles"] <= 5].groupby("payment_type")[["fare"]].mean()

CPU times: user 1.72 s, sys: 682 ms, total: 2.4 s
Wall time: 2.41 s


In [11]:
results_pl

payment_type,fare
str,f64
"""No Charge""",8.901794
"""Unknown""",12.110854
"""Prepaid""",7.254213
"""Mobile""",8.465049
"""Split""",8.457831
"""Credit Card""",9.387332
"""Cash""",8.520036
"""Way2ride""",6.847222
"""Dispute""",9.858734


In [ ]:
results_pd

<html>
<body>
    <div style="display: flex;">
            <a href="https://medium.com/@jgrygolec" style="width: 45em; height: 15em; background-color: white; padding-left: 1em; display: flex; flex-direction: column; justify-content: center; color: inherit; text-decoration: none;">
            <p style="margin: 0; font-size: 1.5em; color: #3D4849;">Thank you for attention!</p>
            <p style="margin: 0; font-size: 1.5em; color: #3D4849; font-weight: bold;">Click to see more of my articles on Medium.</p>
            <p style="margin: 0; font-size: 1.5em; color: #3D4849;">BR,</p>
            <p style="margin: 0; font-size: 1.5em; color: #3D4849;">Jarek Grygolec</p>
        </a>
        <a href="https://medium.com/@jgrygolec" style="width: 30em; height: 15em; background-repeat: no-repeat; background-size: cover; background-position: center;">
                    <img src="https://miro.medium.com/v2/resize:fit:3992/0*WCQwsoZC0FA2-haq" style="width: 100%; height: 100%; object-fit: cover;"></a>
    </div>
</body>
</html>